## **Introduction (1 point)**

In [58]:
%%capture
!python3 -m pip install datasets transformers bertopic

### 1. We choose to focus on Hate because a company providing a plateform could get a lot more problem allowing hate speech (legally speaking) compared to offensive speech. Also a Hate speech is way more specific and easily detected than offensive speech due to the lack of irony for example.

## **Evaluating the dataset (5 points)**

In [59]:
from datasets import load_dataset
ds_builder = load_dataset("tweet_eval", "hate")

ds_builder

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2970
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})

In [60]:
ds_builder['train']['text'][:5]

['@user nice new signage. Are you not concerned by Beatlemania -style hysterical crowds crongregating on you…',
 'A woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot 😎',
 '@user @user real talk do you have eyes or were they gouged out by a rapefugee?',
 'your girlfriend lookin at me like a groupie in this bitch!',
 'Hysterical woman like @user']

In [61]:
ds_builder['train']['label'][:5] #Looking at the labels

[0, 1, 1, 1, 0]

In [62]:
### 2. Nous sommes sur de 3/4 de données utilisées pour le training, 1/4 pour le testing et 1000 données utilisé pour la validation. On remarque également que les labels sont binaire.

### 2. We have a ratio of 3/4 of datas use for training and 1/4 for testing. We also have 1000 datas used independently as a validation set. We observe that the labels are binary values.

In [63]:
from bertopic import BERTopic
from umap import UMAP

umap_model = UMAP(random_state=42)

docs_train = ds_builder['train']['text']
docs_test = ds_builder['test']['text']
docs_validation = ds_builder['validation']['text']

#Create a model for each split of the dataset to see their topics
topic_model_train = BERTopic(embedding_model="all-MiniLM-L6-v2", umap_model=umap_model)
topic_model_test = BERTopic(embedding_model="all-MiniLM-L6-v2", umap_model=umap_model)
topic_model_validation = BERTopic(embedding_model="all-MiniLM-L6-v2", umap_model=umap_model)

topics_train, probs_train = topic_model_train.fit_transform(docs_train)
topics_test, probs_test = topic_model_test.fit_transform(docs_test)
topics_validation, probs_validation = topic_model_validation.fit_transform(docs_validation)

loading configuration file /root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/config.json
Model config BertConfig {
  "_name_or_path": "/root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/pytorch_model.bin
All model checkpoint weights were used when i

In [64]:
topic_model_train.get_topic_info() #Topic infos of the train split

,Topic,Count,Name
0,-1,3722,-1_the_to_and_in
1,0,339,0_bitch_cunt_hole_whore
2,1,315,1_rape_women_woman_raped
3,2,312,2_bitch_cunt_whore_ass
4,3,263,3_men_all_not_women
...,...,...,...
108,107,10,107_mike_labor_supply_bono
109,108,10,108_germany_austrians_jesus_share
110,109,10,109_classes_english_mandy_manning
111,110,10,110_detention_geo_company_parents


In [65]:
topic_model_train.get_topic(0) #Topic of the train split

[('bitch', 0.030274462843129766),
 ('cunt', 0.029124562018841782),
 ('hole', 0.024092793918933854),
 ('whore', 0.02374554807145891),
 ('you', 0.021896775515138366),
 ('user', 0.02127304283650508),
 ('ass', 0.019580679206334422),
 ('fuck', 0.018592282609984655),
 ('shut', 0.018138829908172727),
 ('pussy', 0.0172114671941582)]

In [66]:
topic_model_train.visualize_topics() #Intertopic distance map

In [67]:
topic_model_train.visualize_barchart()

In [68]:
topic_model_test.get_topic_info() #Topic infos of the test split

,Topic,Count,Name
0,-1,40,-1_bitch_and_that_to
1,0,1517,0_user_the_to_buildthatwall
2,1,1227,1_bitch_you_hoe_me
3,2,93,2_shoes_bitch_you_shoe
4,3,32,3_birthday_happy_you_bitch
5,4,29,4_rape_bitch_raped_her
6,5,21,5_new_whole_hair_bitch
7,6,11,6_rape_raped_bitch_dairy


In [69]:
topic_model_test.get_topic(0) #Topic of the test split

[('user', 0.08028526759811937),
 ('the', 0.052907749781593995),
 ('to', 0.045200990582954695),
 ('buildthatwall', 0.035370817129830194),
 ('of', 0.03152085377501743),
 ('and', 0.030703187847265986),
 ('is', 0.03014811904995562),
 ('maga', 0.029947846984035124),
 ('in', 0.02964569237579174),
 ('are', 0.026439499641080708)]

In [70]:
topic_model_test.visualize_topics() #Intertopic distance map

In [71]:
topic_model_test.visualize_barchart()

In [72]:
topic_model_validation.get_topic_info() #Topic infos of the validation split

,Topic,Count,Name
0,-1,63,-1_you_the_to_about
1,0,493,0_to_the_user_in
2,1,224,1_you_bitch_pussy_user
3,2,95,2_rape_women_user_woman
4,3,92,3_hysterical_woman_user_you
5,4,19,4_booker_hysterical_woman_most
6,5,14,5_kitchen_stay_the_in


In [73]:
topic_model_validation.get_topic(0) #Topic of the validation split

[('to', 0.05836973098008722),
 ('the', 0.05474201610815282),
 ('user', 0.05172867822273751),
 ('in', 0.04310498719433161),
 ('of', 0.040380138645850425),
 ('and', 0.0380706891846687),
 ('refugees', 0.03354523860685161),
 ('is', 0.03244484782274974),
 ('immigrant', 0.025983451516675898),
 ('for', 0.025888322827826986)]

In [74]:
topic_model_validation.visualize_topics() #Intertopic distance map

In [75]:
topic_model_validation.visualize_barchart()

In [76]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

loading configuration file /root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/config.json
Model config BertConfig {
  "_name_or_path": "/root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/pytorch_model.bin
All model checkpoint weights were used when i

### 3. The topics of test split and validation split are very similar while there is more topics inside the train split they are still similar between themself. A model trained on those data would likely be stuck on specific topics and not be able to generalize due to the fact that a lot of those topics are similar.

## **Evaluate a model (5 points)**

In [77]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, DataCollatorWithPadding, Trainer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import torch

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='hate'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
data_collector = DataCollatorWithPadding(tokenizer=tokenizer)

labels = ds_builder['test']['label']

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

#text = ds_builder.map(lambda batch: batch['text'], batched=True) #[preprocess(text) for text in ds_builder['train']['text']]
#print(text)
encoded_input = ds_builder.map(lambda batch: tokenizer(batch["text"], padding=True), batched=True, remove_columns=["text", "label"]) #tokenizer(text, return_tensors='pt', padding=True, truncation=True)

#print(encoded_input['input_ids'])

trainer = Trainer(
    model, #Our model
    #training_args, #Specidifed arguments for training
    #train_dataset=tokenized_train_split, #Training tokenized data
    #eval_dataset=encoded_input, #Validation tokenized data
    data_collator=data_collector, #Data Collector object we defined
    tokenizer=tokenizer, #Tokenizer object we defined
)

output = trainer.predict(encoded_input['test'])

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file cardiffnlp/twitter-roberta-base-hate/config.json
Model config RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-hate",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": tru

In [78]:
import torch.nn.functional as F

print(output.predictions)

out = torch.tensor(output.predictions)

predict = F.softmax(out, dim=1)

predict_label = torch.argmax(predict, dim=1)

m = torch.sum(predict_label == torch.tensor(labels))

print(m/len(predict))

[[-1.0485643   1.0212997 ]
 [ 0.46829826 -0.5080502 ]
 [-0.8273667   0.5418106 ]
 ...
 [-0.5798488   0.522204  ]
 [-0.49178636  0.40129316]
 [-1.3835905   1.4396809 ]]
tensor(0.5768)


In [79]:
from sklearn.metrics import classification_report

classification_report(predict_label, labels)

'              precision    recall  f1-score   support\n\n           0       0.31      0.89      0.46       591\n           1       0.95      0.50      0.65      2379\n\n    accuracy                           0.58      2970\n   macro avg       0.63      0.69      0.55      2970\nweighted avg       0.82      0.58      0.61      2970\n'

In [80]:
predict_label_sorted_i = torch.argsort(torch.tensor([max(x[0], x[1]) for x in predict]))[-50::]

predict_label_sorted = predict[predict_label_sorted_i[-50::]]

predict_label_sorted

tensor([[0.0141, 0.9859],
        [0.0141, 0.9859],
        [0.0141, 0.9859],
        [0.0140, 0.9860],
        [0.0140, 0.9860],
        [0.0140, 0.9860],
        [0.0139, 0.9861],
        [0.0139, 0.9861],
        [0.0139, 0.9861],
        [0.0139, 0.9861],
        [0.0138, 0.9862],
        [0.0138, 0.9862],
        [0.0138, 0.9862],
        [0.0138, 0.9862],
        [0.0137, 0.9863],
        [0.0137, 0.9863],
        [0.0137, 0.9863],
        [0.0136, 0.9864],
        [0.0135, 0.9865],
        [0.0135, 0.9865],
        [0.0135, 0.9865],
        [0.0135, 0.9865],
        [0.0135, 0.9865],
        [0.0134, 0.9866],
        [0.0133, 0.9867],
        [0.0133, 0.9867],
        [0.0133, 0.9867],
        [0.0132, 0.9868],
        [0.0131, 0.9869],
        [0.0131, 0.9869],
        [0.0130, 0.9870],
        [0.0130, 0.9870],
        [0.0129, 0.9871],
        [0.0129, 0.9871],
        [0.0127, 0.9873],
        [0.0126, 0.9874],
        [0.0125, 0.9875],
        [0.0123, 0.9877],
        [0.0

In [81]:
predict_label_sorted_i

tensor([ 617, 1298, 2308,  107,  247, 1143,  230, 1750, 1060,  286, 1827,  872,
        2079, 2769,  379,  233, 2664,  137, 2327, 1580,  939, 2463,   64, 2921,
        1996,   98, 1380,  792, 1275,  727, 1482,  412, 1136,  535, 2075,  831,
        2367, 1368, 1052, 1645, 1749,  531,   12, 1972, 2774, 2052, 1109,  582,
         407, 1947])

In [82]:
most_confidents = [ds_builder['test']['text'][i] for i in predict_label_sorted_i]

most_confidents

['Is she admitting to a crime here? Sure sounds like it to me, traitor who should be in prison for life @user @user #BuildTheWall #EndDACA #BoycottNFL #BoycottNike #LockThemAllUp',
 'Enough! #BuildTheWall #DeportThemAll Send the criminals BACK to their country of origin (once we have a GREAT big WALL!)',
 '@user Please DO NOT listen to lame duck @user immigration/The Wall is way more important than tax 2.0! Stick to your instinct #BuildThatWall #BuildTheDamnWall #DeportThemAll',
 'WE AS A NATION DESERVE A REASONABLE EXPECTATION OF SAFETY IN OUR OWN COUNTRY! VOTE REPUBLICAN #BuildThatWall #EndChainMigration #ENDVISALOTTERY #EVERIFY #ENDVOTERFRAUD',
 '@user Just deport all these illegal aliens and their kids back to their 3rd world nation. #MAGA #BuildTheWall #NoDACA #NoAmnesty #DeportAllOfThem',
 'Yes, why is it not closed down and everyone in it deported, they are not our friends, invaders who do not assimulate into our country, time for them to go @user @user #BuildTheWall #EndDACA #B

In [83]:
predict_label_sorted_i = torch.argsort(torch.tensor([x[0] for x in predict]))[-50::]

predict_label_sorted = predict[predict_label_sorted_i]

predict_label_sorted

tensor([[0.9353, 0.0647],
        [0.9354, 0.0646],
        [0.9377, 0.0623],
        [0.9380, 0.0620],
        [0.9388, 0.0612],
        [0.9400, 0.0600],
        [0.9400, 0.0600],
        [0.9406, 0.0594],
        [0.9408, 0.0592],
        [0.9412, 0.0588],
        [0.9415, 0.0585],
        [0.9428, 0.0572],
        [0.9433, 0.0567],
        [0.9435, 0.0565],
        [0.9436, 0.0564],
        [0.9436, 0.0564],
        [0.9443, 0.0557],
        [0.9456, 0.0544],
        [0.9456, 0.0544],
        [0.9470, 0.0530],
        [0.9487, 0.0513],
        [0.9508, 0.0492],
        [0.9541, 0.0459],
        [0.9543, 0.0457],
        [0.9550, 0.0450],
        [0.9553, 0.0447],
        [0.9588, 0.0412],
        [0.9590, 0.0410],
        [0.9592, 0.0408],
        [0.9598, 0.0402],
        [0.9621, 0.0379],
        [0.9625, 0.0375],
        [0.9629, 0.0371],
        [0.9630, 0.0370],
        [0.9646, 0.0354],
        [0.9656, 0.0344],
        [0.9667, 0.0333],
        [0.9714, 0.0286],
        [0.9

In [84]:
most_neutral = [ds_builder['test']['text'][i] for i in predict_label_sorted_i]

most_neutral

["I LOVE WATCHING YOUTUBE VIDEOS WHERE THEY CALL OUT YOUTUBERS OR WHOEVER. I DEADASS LOVE CALL OUT CULTURE, IT'S PETTY BUT BITCH AND?",
 "The Atlantic takes a close look at how programmes for refugees in America are changing for the worse under President Trump's administration. #USA #world #Refugees #RefugeesWelcome #refugeesNOTwelcome #symmitree #4change",
 "I'm glad I have friends who call each other queens and goddesses not bitch or whore.     #goalsreached",
 'Force yourself to love yourself! Love it!! Be it!! Feel it!! Slay it!! Get into your kunt!!!',
 '8. I REMEMBER LIVE DMING U DWP. ur a g and i hope yr 11 treats u right. REMEMBER MY ADIVCE U BITCH ASS CUNT. ilysm!!',
 'BITCH MINSEOK CENTER IS SO DAMN BEAUTIFUL A WHOLE MAIN DANCER',
 'i rlly rlly rlly rlly rlly wanna buy those puma shoes but bITCH I JUST BOUGHT AN ALBUM AND I HAVE LIKE 50 PESOS IN MY WALLET',
 "Trump is a wicked son of a bitch. Can't get his way, so why not hurt everyone. Whoever said he was the best 4🇺🇸was…",


In [85]:
predict_label_sorted_i = torch.argsort(torch.tensor([abs(x[0] - x[1]) for x in predict]))[:50:]

predict_label_sorted = predict[predict_label_sorted_i]

predict_label_sorted

tensor([[0.4993, 0.5007],
        [0.5010, 0.4990],
        [0.4985, 0.5015],
        [0.4982, 0.5018],
        [0.4981, 0.5019],
        [0.5019, 0.4981],
        [0.5024, 0.4976],
        [0.5025, 0.4975],
        [0.4965, 0.5035],
        [0.4963, 0.5037],
        [0.5038, 0.4962],
        [0.5039, 0.4961],
        [0.4958, 0.5042],
        [0.5043, 0.4957],
        [0.5052, 0.4948],
        [0.4947, 0.5053],
        [0.4946, 0.5054],
        [0.4944, 0.5056],
        [0.5059, 0.4941],
        [0.4937, 0.5063],
        [0.5065, 0.4935],
        [0.5065, 0.4935],
        [0.5069, 0.4931],
        [0.5073, 0.4927],
        [0.5075, 0.4925],
        [0.5078, 0.4922],
        [0.5080, 0.4920],
        [0.4918, 0.5082],
        [0.4916, 0.5084],
        [0.4914, 0.5086],
        [0.5086, 0.4914],
        [0.4913, 0.5087],
        [0.4912, 0.5088],
        [0.4907, 0.5093],
        [0.4897, 0.5103],
        [0.4892, 0.5108],
        [0.5108, 0.4892],
        [0.4891, 0.5109],
        [0.5

In [86]:
most_uncertain = [ds_builder['test']['text'][i] for i in predict_label_sorted_i]

most_uncertain

['@user @user . And until Trump made it an issue also the aim of most Politicians - cheap labor & votes for the elites. Most still refuse to support reform. #StopImmigration',
 "y'all ... I came to the gym all excited to see my trainer and he says why are you wearing chanclas? A bitch forgot to switch shoes. 😒☹️",
 "Okay you know what you Chabby fans can't bitch that Gabi is over everyday when that whore Abigail has been on for 10 days straight #Days",
 'Boy, bye😌 "@diorwhore Delete his number and put a face mask on bitch it\'s recovery szn"',
 'This is a couple who just moved on in to Australia and thought if we behave ourselves and involve ourselves in the Community, and have a child here then we will be allowed to stay.Sorry - if that got you in we would be swamped with MILLIONS just like you.#SendThemBack.',
 'Love that u used WOMEN instead of bitch.',
 '@user @user @user Dude your not showing your face so why should I bitch',
 'im that bitch who goes thru the whole comment section

## **Annotate data (7 points)**

In [87]:
!wget  https://raw.githubusercontent.com/mvonwyl/epita/master/NLP/2022/08/tweets.json

--2022-11-22 22:04:45--  https://raw.githubusercontent.com/mvonwyl/epita/master/NLP/2022/08/tweets.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2107788 (2.0M) [text/plain]
Saving to: ‘tweets.json.1’

tweets.json.1       100%[===================>]   2.01M  --.-KB/s    in 0.01s   

2022-11-22 22:04:45 (211 MB/s) - ‘tweets.json.1’ saved [2107788/2107788]



In [88]:
json_data = load_dataset('json', data_files='tweets.json')

  0%|          | 0/1 [00:00<?, ?it/s]

In [89]:
json_data

DatasetDict({
    train: Dataset({
        features: ['id', 'id_str', 'text', 'lang', 'created_at'],
        num_rows: 10000
    })
})

In [90]:
json_data['train'][:3]

{'id': [1410492618790817793, 1410492618769780742, 1410492618790686720],
 'id_str': ['1410492618790817793',
  '1410492618769780742',
  '1410492618790686720'],
 'text': ['YOU BETTER SUCK HIS DICK KOZY I SEE YOU WITH KNUCKLES GET EM GYAAAAL',
  'I still canr believe it.😭😭😭😭😭',
  'You should raise the webform....how would they know then that you completed ur medicals'],
 'lang': ['en', 'en', 'en'],
 'created_at': ['Thu Jul 01 06:57:00 +0000 2021',
  'Thu Jul 01 06:57:00 +0000 2021',
  'Thu Jul 01 06:57:00 +0000 2021']}

In [91]:
encoded_input = json_data.map(lambda batch: tokenizer(batch["text"], padding=True), batched=True, remove_columns=["text", "created_at"])

output = trainer.predict(encoded_input['train'])

  0%|          | 0/10 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: id, lang, id_str. If id, lang, id_str are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10000
  Batch size = 8


In [92]:
print(output.predictions)

out = torch.tensor(output.predictions)

predict = F.softmax(out, dim=1)

predict_label = torch.argmax(predict, dim=1)

[[-0.33326095  0.07276524]
 [ 1.74078    -1.485999  ]
 [ 1.6898811  -1.6296374 ]
 ...
 [ 1.608491   -1.5347848 ]
 [ 1.2212311  -1.1041771 ]
 [ 1.5878689  -1.4942871 ]]


In [93]:
predict_label_sorted_i = torch.argsort(torch.tensor([x[1] for x in predict]))[-20::]

predict_label_sorted = predict[predict_label_sorted_i]

predict_label_sorted

tensor([[0.1977, 0.8023],
        [0.1929, 0.8071],
        [0.1929, 0.8071],
        [0.1725, 0.8275],
        [0.1569, 0.8431],
        [0.1488, 0.8512],
        [0.1486, 0.8514],
        [0.1457, 0.8543],
        [0.1359, 0.8641],
        [0.1318, 0.8682],
        [0.1250, 0.8750],
        [0.1206, 0.8794],
        [0.1179, 0.8821],
        [0.1010, 0.8990],
        [0.0893, 0.9107],
        [0.0810, 0.9190],
        [0.0727, 0.9273],
        [0.0695, 0.9305],
        [0.0679, 0.9321],
        [0.0608, 0.9392]])

In [94]:
hateful_20 = [json_data['train']['text'][i] for i in predict_label_sorted_i]

hateful_20

['The law of NATURE? Where there is a need for a FEMALE and a MALE to procreate. That might be why...',
 'This issa lie! Black women don’t cheat.',
 'Suck on this mf',
 'The sooner you realise vagina runs the world the better!!',
 'DAMN U DIDNT HAVE TO BE THAT AGGRESSIVE WHORE',
 'Stop having side chicks, this creates more problems for you...solve your issues at home',
 'This is why women don’t like #you',
 'Only a woman that is interested for material things and money says she agrees with her man doing anything he wants. If she loves you, she gonna slap you and broke your head till you make it right!',
 'can’t keep a lil hoe',
 'Better body then most woman on here',
 'One thing bout me we all gon laugh in this bitch🤣',
 'I wish more ladies would be self obsessed! So hard to get people to see the worth in themselves!',
 'Returning bitches to they last owner',
 "You're a little bitch",
 'How many bitches you gone have wearing the same number ?😂',
 'Lmaoooo bitches b moving different',
 

In [95]:
import random

x = 0

random_text = set()
n = len(predict)
while x < 80:
  i = random.randint(0, n)
  if i in predict_label_sorted_i or json_data['train']['text'][i] in random_text:
    continue

  random_text.add(json_data['train']['text'][i])
  x += 1

random_text

{'"Wow, what a great presentation Mr Akihiko. Thank you for that. Ladies and gentlemen please let\'s have another round of applause for our first speaker," Aquillon flashed a charming smile at the crowd and clapped along side them. \n\n"And those prizes sure were worth it," he laughs.',
 '... because you can simply assign sub-trees to other symbolic execution instances. To his experience, the sub-trees are in practice large enough to avoid thinking about balancing the workload between the parallel instances.',
 '@user\n HACKED BY MC CAVERINHA EZZZ LOL XDDD 🧔🏿🧔🏿🧛🏿\u200d♀️🧛🏿\u200d♀️🧛🏿\u200d♀️🧛🏿\u200d♀️🧛🏿\u200d♀️🧜🏿🧜🏿🧜🏿🧜🏿\u200d♂️🧜🏿\u200d♂️🧜🏿\u200d♂️🧜🏿\u200d♂️🦸🏿\u200d♀️🦸🏿\u200d♀️🦸🏿\u200d♀️',
 'Appears GETTR is under attack - we can probably count the minutes until a full crash. Just read the ToS - humorous.',
 'BUY indusind cmp - 1015.15 sl - 1008',
 'Bernard manning joke lol',
 'Brittney Spears needs a Supreme Court to throw out the lower court s decision!!!!',
 "Can't visit",
 'Clarence T

In [96]:
tweet_100 = np.concatenate([hateful_20 ,list(random_text)])

tweet_100

array(['The law of NATURE? Where there is a need for a FEMALE and a MALE to procreate. That might be why...',
       'This issa lie! Black women don’t cheat.', 'Suck on this mf',
       'The sooner you realise vagina runs the world the better!!',
       'DAMN U DIDNT HAVE TO BE THAT AGGRESSIVE WHORE',
       'Stop having side chicks, this creates more problems for you...solve your issues at home',
       'This is why women don’t like #you',
       'Only a woman that is interested for material things and money says she agrees with her man doing anything he wants. If she loves you, she gonna slap you and broke your head till you make it right!',
       'can’t keep a lil hoe', 'Better body then most woman on here',
       'One thing bout me we all gon laugh in this bitch🤣',
       'I wish more ladies would be self obsessed! So hard to get people to see the worth in themselves!',
       'Returning bitches to they last owner', "You're a little bitch",
       'How many bitches you gone have 